In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point
from scipy.spatial import ConvexHull
from scipy.ndimage import maximum_filter, gaussian_filter
from scipy.interpolate import RegularGridInterpolator


def get_contour_lines(cs):
    """
    Extract contour lines from a matplotlib QuadContourSet.
    Each contour is returned as a dict with 'v', 'x', 'y'.
    """
    contours = []
    for level, segs in zip(cs.levels, cs.allsegs):
        for seg in segs:
            if len(seg) > 2:  # ignore very short segments
                x, y = seg[:, 0], seg[:, 1]
                contours.append({"v": level, "x": x, "y": y})
    return contours



def is_contour_convex(x, y, deficiency_thresh=5.0):
    """Check convexity of a contour by comparing polygon area to convex hull area."""
    pts = np.column_stack((x, y))
    poly = Polygon(pts)
    if not poly.is_valid:
        return False
    area = poly.area
    hull_area = ConvexHull(pts).volume  # area in 2D
    deficiency = abs(hull_area - area) / area * 100
    return deficiency <= deficiency_thresh


def find_2d_peak(VMatrix, x, y, mode="maxima"):
    """Find local maxima or minima of a 2D scalar field."""
    if mode.lower() == "maxima":
        interp_obj = RegularGridInterpolator((x, y), VMatrix.T, method="linear", bounds_error=False, fill_value=None)
    elif mode.lower() == "minima":
        interp_obj = RegularGridInterpolator((x, y), -VMatrix.T, method="linear", bounds_error=False, fill_value=None)
    else:
        raise ValueError("mode must be 'maxima' or 'minima'")

    dx = abs(x[1] - x[0])
    dy = abs(y[1] - y[0])
    xi, yi = np.meshgrid(x, y, indexing="xy")
    z1, z2 = np.gradient(VMatrix, dx, dy)

    # simple local maxima detection using max filter
    neighborhood = maximum_filter(VMatrix, size=3)
    mask = (VMatrix == neighborhood)
    yp, xp = np.where(mask)
    vals = VMatrix[yp, xp]

    return x[xp], y[yp], vals


def ContourExtraction(VMatrix, xi, yi, Nct, MinLength, DeficiencyThresh):
    x = xi[0, :]
    y = yi[:, 0]

    # Step 1: Extract closed, convex contours
    cs = plt.contour(x, y, VMatrix, Nct)
    s = get_contour_lines(cs)
    print(f"... {len(s)} contours are extracted.")

    bnd = {"xc": [], "yc": [], "cval": [], "xp": [], "yp": [], "valp": []}

    for contour in s:
        cx, cy = contour["x"], contour["y"]
        if (cx[0] == cx[-1]) and (cy[0] == cy[-1]):  # closed check
            length = np.sum(np.sqrt(np.diff(cx) ** 2 + np.diff(cy) ** 2))
            if length > MinLength:
                if is_contour_convex(cx, cy, DeficiencyThresh):
                    bnd["xc"].append(cx)
                    bnd["yc"].append(cy)
                    bnd["cval"].append(contour["v"])

    print(f"... {len(bnd['xc'])} contours are closed and convex.")

    # Step 2: Find local maxima
    xp, yp, valp = find_2d_peak(VMatrix, x, y, mode="maxima")

    in_max = []
    for cx, cy in zip(bnd["xc"], bnd["yc"]):
        mask = [Point(px, py).within(Polygon(np.column_stack((cx, cy)))) for px, py in zip(xp, yp)]
        in_max.append(mask)
    in_max = np.array(in_max, dtype=bool)

    n_in_max = np.sum(in_max, axis=1)
    ind_eliminate_1 = n_in_max != 1

    for key in ["xc", "yc", "cval"]:
        bnd[key] = [val for i, val in enumerate(bnd[key]) if not ind_eliminate_1[i]]

    valid_indices = np.where(~ind_eliminate_1)[0]
    indmax = []
    for i, row in enumerate(in_max[~ind_eliminate_1, :]):
        indmax.extend(np.where(row)[0])

    bnd["xp"] = xp[indmax].tolist()
    bnd["yp"] = yp[indmax].tolist()
    bnd["valp"] = valp[indmax].tolist()

    # Step 3: Keep only outermost contours
    nct_filt2 = len(bnd["xc"])
    if nct_filt2 > 0:
        xq = [c[0] for c in bnd["xc"]]
        yq = [c[0] for c in bnd["yc"]]
    else:
        return bnd

    ind_eliminate_2 = np.zeros(nct_filt2, dtype=bool)
    for ii in range(nct_filt2):
        poly = Polygon(np.column_stack((bnd["xc"][ii], bnd["yc"][ii])))
        for j, (qx, qy) in enumerate(zip(xq, yq)):
            if ii != j and poly.contains(Point(qx, qy)):
                ind_eliminate_2[j] = True

    for key in ["xc", "yc", "cval", "xp", "yp", "valp"]:
        bnd[key] = [val for i, val in enumerate(bnd[key]) if not ind_eliminate_2[i]]

    return bnd

In [ ]:
from utils_mitgcm import open_mitgcm_ds_from_config

In [ ]:
model = 'geneva_dummy_extended'

params_file = 'swirl_05'

# Open MITgcm results

In [ ]:
mitgcm_config, ds_mitgcm = open_mitgcm_ds_from_config('../config.json', model)

# Select depth and time

In [ ]:
depth_indices = [0]#range(len(ds_mitgcm.Z.values))
time_indices = [-1]#range(len(ds_mitgcm.time.values))

# Run LAVD

In [ ]:
data_snapshot = ds_mitgcm.isel(time=-1, Z=0)

x = np.array(range(0,len(data_snapshot['XC'])))*200 + 100
y = np.array(range(0,len(data_snapshot['YC'])))*200 + 100
xi, yi = np.meshgrid(x, y, indexing='xy')

u = data_snapshot['UVEL'].values
v = data_snapshot['VVEL'].values
u = np.where(np.isnan(u), 0, u)
v = np.where(np.isnan(v), 0, v)
dx = data_snapshot.dxC.values[0][0]
dy = data_snapshot.dyC.values[0][0]

# Compute vorticity as scalar field
dx = x[1] - x[0]
dy = y[1] - y[0]
dv_dx = np.gradient(v, dx, axis=1)
du_dy = np.gradient(u, dy, axis=0)
vorticity = dv_dx - du_dy


# Contour extraction parameters
Nct = 20
MinLength = 5
DeficiencyThresh = 5


# Run contour extraction
bnd = ContourExtraction(vorticity, xi, yi, Nct, MinLength, DeficiencyThresh)

In [ ]:
plt.plot(bnd['xp'], bnd['yp'], 'bo', markersize=6)

In [ ]:
# Plot results
plt.close('all')
plt.figure(figsize=(16,5))
plt.contour(xi, yi, vorticity, Nct, colors='lightgray')
for xc, yc in zip(bnd['xc'], bnd['yc']):
    plt.plot(xc, yc, 'r-', linewidth=2)
    plt.plot(bnd['xp'], bnd['yp'], 'bo', markersize=1)

plt.title('Vortex Contours and Centers from u,v')
plt.xlabel('x')
plt.ylabel('y')
plt.axis('equal')
plt.show()